# Chains
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Chains allow us to combine multiple components together to create a single, coherent application.

In [ ]:
!pip install python-dotenv chromadb langchain openai

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["service"],
    template="What is a good name for a company that provides {service}?",
)

We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM.

In [2]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("SaaS solutions"))



1. CloudConnect Solutions
2. CloudCatalyst
3. SaaS-Tech
4. CloudMinds
5. SimpliCloud Services
6. CloudBoost Solutions
7. Skyway Solutions
8. Nimbus Technologies
9. CloudFactory
10. StreamTech Solutions


If there are multiple variables, you can input them all at once using a dictionary.

In [3]:
prompt = PromptTemplate(
    input_variables=["company", "service"],
    template="What is a good name for a {company} that provides {service}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'company': "AI Startup",
    'service': "cloud services and automation"
    }))



Cloud Autom8.


You can use a chat model in an `LLMChain` as well:

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a {company} that provides {service}?",
            input_variables=["company", "service"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.8)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run({
    'company': "Real Estate Agency",
    'service': "provides social media automation"
    }))

SocialEstate Automation


Adding memory (state)

Chains can be initialized with a Memory object, which will persist data across calls to the chain. This makes a Chain stateful.

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
# -> The first three colors of a rainbow are red, orange, and yellow.
conversation.run("And the next 4?")
# -> The next four colors of a rainbow are green, blue, indigo, and violet.

'The next four colors of a rainbow are green, blue, indigo, and violet.'

Saving a chain to disk

In [7]:
# conversation.save("./chains/06_langchain_chains.json") 
# ValueError: Saving of memory is not yet supported.

chain.save("./chains/06_langchain_chains.json")

Loading chain from a disk

In [8]:
from langchain.chains import load_chain

# chain = load_chain("./chains/06_langchain_chains.json")
# ValueError: Loading openai-chat LLM not supported

ValueError: Loading openai-chat LLM not supported

In [10]:

print(chain.run({
    'company': "Windows manufacturer",
    'service': "sells quality alluminium windows"
    }))

AluWinCo


There are additional ways of running LLM Chain.

`apply` allows you run the chain against a list of inputs:

In [11]:
input_list = [
    {'company': "Windows manufacturer",'service': "sells quality alluminium windows"},
    {'company': "Computer store",'service': "sells all kinds of electronics"},
]

chain.apply(input_list)

[{'text': 'Aluview Windows'}, {'text': 'Tech Haven'}]

`generate` is similar to apply, except it return an LLMResult instead of string. LLMResult often contains useful generation such as token usages and finish reason.

In [12]:
chain.generate(input_list)

LLMResult(generations=[[ChatGeneration(text='AluWinTech', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='AluWinTech', additional_kwargs={}, example=False))], [ChatGeneration(text='Tech Haven', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='Tech Haven', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 48, 'completion_tokens': 6, 'total_tokens': 54}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('1badb667-0ff0-445f-8814-0e03c969e982')), RunInfo(run_id=UUID('2cfa9515-c80e-4b15-8307-c68766de21a1'))])

Parsing the outputs


By default, LLMChain does not parse the output even if the underlying `prompt` object has an output parser. If you would like to apply that output parser on the LLM output, use `predict_and_parse` instead of `predict` and `apply_and_parse` instead of `apply`.

In [13]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
prompt = PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict()

'\n\nRed, orange, yellow, green, blue, indigo, and violet.'

In [14]:
# With predict_and_parse:
llm_chain.predict_and_parse()

/home/azureuser/miniconda3/envs/RemoteEnv/lib/python3.8/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']